# Store and Query OpenAI Embeddings in Azure SQL DB (T-SQL)

## Vector Similarity Search in SQL DB

Learn how to store and query vector embeddings in Azure SQL Database using T-SQL with this step-by-step tutorial.

# Dataset

- For this tutorial we use the sample data [finefoodembeddings.csv](https://github.com/Azure-Samples/azure-sql-db-vector-search/blob/a181e15337402e568f4fc66fe5941e5973171972/VectorSearch_Notebooks/Datasets/finefoodembeddings.csv) The dataset has been created using the customer reviews from FineFoods Amazon and enriching it with embeddings generated via the `text-embedding-small` Azure OpenAI model. The embeddings have been generated using the concatenation of Product summary and Customer Reviews (Summary + Text)

# Prerequisites

- **Azure Subscription**: [Create one for free](https:\azure.microsoft.com\free\cognitive-services?azure-portal=true)
    
- **Azure SQL Database**: [Set up your database for free](https:\learn.microsoft.com\azure\azure-sql\database\free-offer?view=azuresql)
    
- **Azure Data Studio**: Download [here](https://azure.microsoft.com/products/data-studio) and run the notebook using [SQL Kernel for Notebooks in Azure Data Studio](https://learn.microsoft.com/azure-data-studio/notebooks/notebooks-guidance#connect-to-a-kernel).
    
- **Azure OpenAI Access**: Make sure you can access OpenAI service by following the documentation here: [How do I get access to Azure OpenAI?](https://learn.microsoft.com/azure/ai-services/openai/overview#how-do-i-get-access-to-azure-openai)
    
- **Azure OpenAI Resource**: Deploy an embeddings model (e.g., `text-embedding-small` or `text-embedding-ada-002`) following the steps in [Create and deploy an Azure OpenAI Service resource](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource)
    
- For this specific sample you have to deploy an embeddings model `text-embedding-small` model, the same used for the source we are using in this sample. Once that is done, you need to get the API KEY and the URL of the deployed model  [Embeddings REST API](https:\learn.microsoft.com\azure\cognitive-services\openai\reference#embeddings).
    

Then retrieve the Azure OpenAI _endpoint_ and _key_:

![Azure OpenAI Endpoint and Key](../Assets/endpoint.png)

Then retrieve the deployed model _name_ from Azure OpenAI Studio

![Deployed OpenAI Models](../Assets/modeldeployment.png)

In the notebook we will use [`sp_invoke_external_rest_endpoint`](https://learn.microsoft.com/sql/relational-databases/system-stored-procedures/sp-invoke-external-rest-endpoint-transact-sql) that will call the OpenAI embedding model you have deployed before

# Importing Data into SQL DB

Import the data into the `finefoodreviews` table by following these steps:

1. Utilize the [SQL Server Import extension](https:\learn.microsoft.com\azure-data-studio\extensions\sql-server-import-extension) in Azure Data Studio to convert `.txt` and `.csv` files into a SQL table.
2. Step by Step instructions are available [here](https:\learn.microsoft.com\azure-data-studio\extensions\sql-server-import-extension)

![ImportWizard](../Assets/importwizard.png)

You can see the preview of the data you will be importing

![ImportPreview](../Assets/wizardpreview.png)

Make sure to modify the data types accordingly before the import process

- **Id**: `bigint` NOT NULL
- **ProductId**: `nvarchar(500)` NULL
- **UserId**: `nvarchar(50)` NULL
- **Time**: `nvarchar` NULL
- **Score**: `bigint` NULL
- **Summary**: `nvarchar` NULL
- **Text**: `nvarchar` NULL
- **Combined**: `nvarchar` NULL
- **Vector**: `nvarchar` NULL

![ImportWizardDatatypes](../Assets/importwizarddatatypes.png)

Once you have successfully inserted the data, we can now try our new Vector functions

![ImportSuccess](../Assets/importsuccess.png)

# Running the Notebook

Execute the notebook using the [SQL Kernel for Notebooks in Azure Data Studio](https://learn.microsoft.com/azure-data-studio/notebooks/notebooks-guidance#connect-to-a-kernel).

Lets take a look at the data in the table. The dataset has been created using the customer 
reviews from FineFoods Amazon and enriching it with embeddings generated via the text-embedding-small OpenAI model.
The embeddings have been generated using the concatenation of Summary + Text


In [8]:
select top(5) * from dbo.finefoodreviews 
order by ID

(5 rows affected)

Total execution time: 00:00:00.591

Id Time ProductId UserId Score Summary Text combined vector 1 1303862400 B001E4KFG0 A3SGXH7AUHU8GW 5 Good Quality Dog Food I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than most. Title: Good Quality Dog Food; Content: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than most. [0.02088439, -0.00022463033, -0.0019172364, -0.01744861, 0.014939199, -0.02193975, -0.013109909, 0.05281488, 0.03412329, -0.01803492, 0.07636112, -0.04174533, -0.019606235, -0.002708756, -0.011679311, 0.043129023, -0.025868034, 0.01561932, -0.04538046, 0.003116242, 0.051501542, -0.005306113, -0.048312012, 0.038016394, -0.00980898, -0.0051859193, -0.02225636, 0.029432803, 0.04125283, -0.0318015, -0.030558519, -0.016932657, -0.00094909064, -0.0075399573, -0.024320172, 0.0035589067, 0.025117556, -0.042964857, 0.04868725, -0.035600793, -0.03295067, 0.033302456, 0.033771504, 0.016158726, 0.058068223, 0.003110379, 0.035366267, 0.0026193436, -0.014786758, -0.0002359901, 0.0110343695, 0.014692949, 0.024531245, 0.033560432, 0.043504264, -0.028752683, 0.021576239, -0.002090198, -0.015982833, 0.019055102, -0.0074989153, -0.0145874135, 0.048358917, 0.020989927, -0.011327525, -0.060413465, -0.07692398, -0.06379062, 0.028400896, -0.0024595738, 0.01397765, 0.010881929, 0.008999871, 0.018539147, -0.019113733, 0.0056901462, -0.022268085, -0.0033712871, 0.054878693, 0.015842117, -0.056801792, 0.057739887, -0.042589616, -0.00997901, -0.027603513, -0.031942215, 0.018363254, -0.008935377, 0.023640051, 0.010278028, 0.046482723, -0.0033800818, -0.03972842, -0.014997831, 0.039071754, 0.008777073, -0.00070833677, 0.042097118, -0.015560689, 0.0007746632, -0.0039282823, 0.022021834, 0.00046062045, 0.044700336, 0.034920674, -0.029925304, 0.039892588, -0.006566681, 0.02178731, -0.027884942, -0.0073054326, 0.004359221, -0.037922584, -0.0063145673, -0.046834506, -0.055769883, -0.013344434, -0.035718054, -0.05717703, -0.010283892, -0.023511063, 0.053190116, -0.016111821, -0.048452727, -0.014810211, -0.015666224, 0.039962944, 0.02088439, 0.0041452176, -0.056473456, 0.00547028, -0.022502609, 0.024929935, -0.027697323, -0.06787134, -0.006396651, -0.04319938, 0.0026061516, -0.023346897, 0.06726158, 0.024601601, -0.055769883, 0.012969195, 0.013719673, 0.034592338, -0.05079797, -0.03693758, -0.03095721, 0.016991287, 0.057130124, -0.029737685, 0.05070416, -0.07847184, 0.026782678, 0.04878106, 0.026430892, 0.021822488, -0.008331477, -0.02910447, 0.039704967, -0.02910447, -0.0057223937, -0.004154012, 9.738256e-05, -0.029034112, 0.0059920964, -0.006900878, -0.012500146, -0.028565062, -0.021201, 0.03414674, 0.009369247, 0.0033243822, -0.034381267, -0.00694192, -0.055488456, 0.0010494963, -0.02525827, 0.030230185, -0.013907293, -0.017307896, 0.029831495, 0.04068997, -0.008841567, -0.04230819, 0.020169092, -0.023980113, 0.035342816, 0.0033155878, 0.031778045, -0.033372812, -0.07589207, 0.0080383215, -0.013215446, 0.020896118, -0.031168282, -0.0026999612, 0.011474103, 0.027603513, 0.012195265, -0.002826018, -0.003333177, -0.0040748604, -0.013848661, -0.0056080627, 0.0031778046, -0.042566165, 0.045427363, -0.016322894, -0.055957504, 0.009926242, 0.002959404, -0.04202676, 0.031684235, 0.024156006, 0.0014950925, -0.0062559363, 0.040807232, 0.027791133, 0.026008748, 0.040947948, -0.0063614724, 0.05516012, -0.010799845, -0.02872923, 0.0027380716, 0.0056022, -0.07457873, 0.006895015, -0.037711512, -0.0061504003, -0.009644813, -0.026196368, 0.0016020943, -0.017190633, 0.025821129, 0.047678795, 0.049812965, 0.054222025, 0.012922291, -0.025094103, 0.021564512, -0.01952415, 0.04760844, 0.033208646, 0.002657

To have the broadest compatibility with any language and platform in the first wave vectors 
will take advantage of existing `varbinary` data type to store vector binary format.

Add the New Column: vector with the type `varbinary(8000)`


In [9]:
ALTER TABLE [dbo].[finefoodreviews]
ADD [VectorBinary] VECTOR(1536);

Commands completed successfully.

Total execution time: 00:00:00.159

Then, update the new column with the transformed data from the embedding column by converting the JSON array to a compact binary representation of a vector

In [10]:
UPDATE [dbo].[finefoodreviews]
SET [VectorBinary] = CAST([vector] AS VECTOR(1536));
GO

(10000 rows affected)

Total execution time: 00:00:12.554

We can drop the previous vector column now as we will be using compact binary representation 
of a vector in the column vectorbinary created before this.

In [ ]:
ALTER TABLE dbo.finefoodreviews
DROP COLUMN vector;

Vectors are stored in an efficient binary format that also enables usage of dedicated CPU vector 
processing extensions like SIMD and AVX.      

In [12]:
select top(5)  
    ID, ProductId, Summary, VectorBinary  
from 
    [dbo].[finefoodreviews]


(5 rows affected)

Total execution time: 00:00:00.186

ID text summary vectorbinary 1 B001E4KFG0 Good Quality Dog Food 0xA901000600000000BE15AB3CBF8A6BB9C74BFBBA63F08EBC8BC3743CFEBAB3BCF2CA56BC6A54583DDDC40B3DF8BD93BC38639C3D27FD2ABD419DA0BC628531BB955A3FBC0FA8303D33E9D3BC2DE87F3CDCE039BDDD394C3B48F3523DE7DEADBBD1E245BD14B71B3DD8B520BCA5EEA9BBF852B6BC101DF13CBAF8283D4A4202BDDC55FABC5BB68ABC65CC78BABF11F7BB193BC7BC8C3C693B55C3CD3CEBFB2FBD486C473D23D211BD48F706BD2868083DFD530A3D4E5F843CF2D86D3D80D74B3B38DC103D4BA92B3B294472BC1C7477B980C9343CB2BA703CC0F5C83CAA76093D8731323DBF8AEBBCA7C0B03CB4FB08BB6EEE82BC72199C3C76B9F5BB0C006FBC0014463D11F3AB3C159739BC1C7477BD528A9DBDA8A482BDFFA8E83CCD3021BB8302653C1E4A323C3274133C69DF973C67949CBC6874BA3B8F6BB6BCD0F05CBB7BC8603D54C7813CFFA868BDA9806C3D73722EBD007F23BCC320E2BCD7D502BD896E963CB06512BCC8A8C13C2D65283CAB643E3D5D845DBB45BA22BD76B975BCB409203DB6CD0F3CADAF39BA076E2C3D43F27EBCC4124B3AD2B880BB2267B43C6E7FF139B317373DFB080F3DE925F5BC6966233D502DD7BB4D7BB23CF66EE4BC6962EFBBCCD78E3BB6541BBD6EEACEBB8AD53FBDF66E64BD9DA25ABC184D12BD76326ABDC57D28BC469AC0BCE1DD593DF0FC83BC5E7646BD87A672BC745680BC2FB0233DBE15AB3C9BD4873BB65067BD0940B33B6457B8BCDD39CC3C7FE5E2BC21008BBDFF9AD1BBD5F130BDF8CB2A3BFE41BFBC70C0893D4C89C93CF66E64BDBF7C543C7BC8603CB2B00D3D881150BDDC4B17BDFA99FDBC50318B3C47016A3D729CF3BC2AAF4F3DD8B5A0BD5967DB3CA6CE473D9985D83C13C5B23CBF8008BC7F6CEEBCADA1223D7F6CEEBCEB82BBBB611E88BBDD39CC38F2D8EDBC5959C43BC320E2BB6ACD4CBC4701EABCB8ADADBC74DD0B3D7881193CE1DD593B5FD30CBD0C79E3BBDD4763BD418F893A6EEACEBC4BA5F73C6ADB63BC4AC98DBC2E61F43C87AA263D38DC10BC5A4B2DBDA739A53CF171C4BCA1C3103D554A593BB229023DEEB108BD4D6D9BBD2AB3033C998558BC562EAB3CA054FFBCD5F130BBE0FD3B3CC320E23CA6CE473CB73439BB6E715ABB688685BB7FE562BCD7C3B7BBB742503BDC592EBD0B123A3D97B785BCB23365BDADA1223C8FF2413B40242CBD54C7013DD1E2C53CFBF6C33A99FECCBB7C25273D3BAAE33C4C10D53C09B9273DE673D0BB94EF613DD5F130BC9059EBBC3871333BA992B73BBCBC98BD94EFE13B63771ABD4C89C9BB47051EBCC499D6BC5DFDD13A5FD38CBCD586D33CD74A433DAE084C3DEA175E3D04B8533C2692CDBC0FA8B03C1CF19FBC1101433DCA05083DAD282E3B5DFDD1BCB89F96BD74DD8BBC4C10D53BD06D053C76B9F53C9AF0353D261959BD076A783DD0F0DC3A73F939BD0C6F00BC562E2BBCE1E18DBA30A20CBD0B12BA3CDE2B353DFA997D3C61A15FBDADAF393DC74FAF3C50B8163D9AE29EBC15A5D03CB650E7BB879C8FBC3F4E71BCF1632D3BF0838F3C43E467BCD5F130BCB7C9DB3C142CDC3CFB045B3D6F63C3BD184D123C929A03BD55B5B6BD3270DF3CFB045BBCD586533BAE08CCBC2C850A3DE7490BBCEA175E3C427DBE3C8389703C0B7D17BDC2322D3DDEB2C03C3BA0803C9A69AA3CB39E423DF7604DBD24393B3D1096E53C55B536BB32F7EA3C54C701BD6EF8653C90596B3DB65067BCF7E7D8BCF7D9413D1F3C1B3DD06D053D91C4C83CECDF01BD378DE13C9897A3BCB65E7EBC6C2A93BC53E32FBC3A316FBC21FCD63CECDF81BBD15BBA3CBE9CB63B881504BCA72B0EBDE83740BC1D5849BCCC4C4FBBDDC40BBC9893EFBC43E81B3D9DB0F1BCABDDB23C990C64BDA2B145BB730751BB3FB94EBCE248B73B7C33BE3B90D693BC151E45BD3E5225BD058E0E3D436FA7BC5D84DD3CDB79103D8B409D3CFF215DBC97A5BA3C27761F3CB233E53CF8441FBA486CC7BC9985D8BB23591DBCE66539BD59EE66BBC499D6BC01718C3B9C37FDBBA1C3903C419D203C0014463D4361903B947A21BB98890CBA729C73BDBAF8283B1F2E843C6E750E3D3C801E3D6A5458BCE248B73CF1EA38BD1DDF543DEA9E693C81BB1D3D18E234BD210EA23CA0547F3D4116153D185775BA34BF8EBCE164E53CF3434B3C219179BD4AB7423D436BF3BC4D6D9BBCFC61213B327493BDA647BCBC53E3AFBBEA09C7BC8898DB39D4946ABD4A42023C2B1A2DBDCA05083C1D58493DF6EB8C3C8CB911BD02D8B53C8551943C97B7853BCEA995BA4788F53C1AA6A4BBBD358DBC8E040D3D59E04F3C6293C83B6CA387BB369F2CBC5198343CEE34603CDBF2843DC74B7BBC7BBA493CA5EE29BD18D0E9BC0B8BAEBBC5F61C3D9CB4A5BBBE23C2BC5A4B2D3D65C215BD094033BD14B71BBDF67298BDFA9D313C167B0B3D595944BC59E403BD671BA8BD2F4546BDCC4CCFBC034DF6BCAAEBC9BC0A2405BDDE1D9EBCED462B3CFFA868BD5EEF3ABDEA175E3C896E163DE9A29DBAD07B1C3B6BBF35BDCBE1713CB2B08D3B444FC53A5BB68ABC728E5CBB6A54583D8302E53B14A904BDEA9E693C5C92F4BCD077E83CFB8B663C486CC73C7C33BEBBD2C6973CC32062BB19C2523CC833813C167B0B3DD07B1CBDD411133D5875F2BCC82FCDBC6DFC99BC033F5F3CBC51BBBC5FD38C3C076EAC3CF25FF93C2948A63C3FCB193C2ABD663CBAF828BD50B8163D8903B9BCA655D33C0C79633CC1CB03BDA39517BDF25FF9

For this specific sample you have to deploy an embeddings model `text-embedding-small` model, the same used for the source we are using in this sample. Once that is done, you need to get the API KEY and the URL of the deployed model  [Embeddings REST API](https:\learn.microsoft.com\azure\cognitive-services\openai\reference#embeddings) as shown in the screenshots in the "Getting Started" part of this notebook

We will invole [`sp_invoke_external_rest_endpoint`](https://learn.microsoft.com/sql/relational-databases/system-stored-procedures/sp-invoke-external-rest-endpoint-transact-sql) in the below store proc that will call the OpenAI embedding model you have deployed before.  To execute the stored procedure, the executing principal must have the `execute external endpoint` permissions: This permission is covered by the **CONTROL** permission, so anyone having **CONTROL** permission on server or database will be able to also execute the stored procedure. More info [here](https:\learn.microsoft.com\sql\relational-databases\system-stored-procedures\sp-invoke-external-rest-endpoint-transact-sql?view=azuresqldb-current&tabs=request-headers)
  

OpenAI models are available as REST endpoints and thus can be easily consumed from Azure SQL Database using the sp\_invoke\_external\_rest\_endpoint system stored procedure: Using a call to a REST service to get embeddings is just one of the integration options you have when working with SQL Database and OpenAI. You can let any of the available models access data stored in Azure SQL Database to create solutions where your users can interact with the data as shown here

In [17]:
create procedure [dbo].[get_embeddings]
(
    @model varchar(max),
    @text nvarchar(max),
    @embedding vector(1536) output
)
as
begin
    declare @retval int, @response nvarchar(max);
    declare @url varchar(max);
    declare @payload nvarchar(max) = json_object('input': @text);

    -- Set the @url variable with proper concatenation before the exec statement
    set @url = 'https://<yourresourcename>.openai.azure.com/openai/deployments/' + @model + '/embeddings?api-version=2023-03-15-preview';

    exec dbo.sp_invoke_external_rest_endpoint 
        @url = @url,
        @method = 'POST',   
        @payload = @payload,   
        @headers = '{"Content-Type":"application/json", "api-key":"<openaikey>"}', 
        @response = @response output;

    -- Use json_query to extract the embedding array directly
    declare @jsonArray nvarchar(max) = json_query(@response, '$.result.data[0].embedding');

    -- Convert JSON array to vector and return it
    set @embedding = cast(@jsonArray as vector(1536));
end
GO


Commands completed successfully.

Total execution time: 00:00:00.158

## Performing a similarity search

Lets now query our embedding table to get the top similar reviews given the User search query.

What we are doing: Given any user search query, we can get the vector representation of that text. Then we can use that vector to calculate the cosine distance against all the customer review comments stored in the database and take only the closest ones which will return the product most likely connect to the product we are interested in. The reviews with the highest similarity are considered the most relevant to the query, helping users discover products or experiences related to their search.
The most common distance is the cosine similarity, which can be calculated quite easily in SQL with the help of the new distance functions

This approach showcases the power of semantic search by finding reviews that are contextually related to the search query, even if they don’t contain exact match keywords  

In [19]:
declare @e vector(1536) 
exec dbo.get_embeddings @model = '<yourmodeldeploymentname>', @text = 'help me find healthy cat food', @embedding = @e output;

SELECT TOP(10) 
       ProductId, 
       Summary, 
       text,
       vector_distance('cosine', @e, vectorbinary) AS distance
FROM 
       dbo.finefoodreviews
ORDER BY 
       distance

(10 rows affected)

Total execution time: 00:00:01.038

ProductId,Summary,text,similarity_score
B003SE19UK,Palatable and healthy,"Before I was educated about feline nutrition, I allowed my cats to become addicted to dry cat food. I always offered both canned and dry, but wish I would have fed them premium quality canned food and limited dry food. I have two 15 year old cats and two 5 year old cats. The only good quality dry foods they will eat are Wellness and Innova. Innova's manufacturer was recently purchased by Procter&Gamble. I began looking for a replacement. After once again offering several samples (from my local holistic pet store) Holistic Select was the only one (other than the usual Wellness and Innova) they would eat. For finicky cats, I recommend trying Holistic Select. It is a good quality food that is very palatable for finicky eaters.",0.6271999928953957
B000VK6NDC,"Highest quality, delicious food for your cats!","My cats both LOVE this food. I've had varying success with about 7 different kinds of canned food. This one is a hit with both. It's natural, mostly organic, doesn't have any junk or fillers, and I have peace of mind knowing that we're giving them high quality food from an ethical company. Since they've been on this diet they are more active, alert and happy. Their coats are so soft and shiny and they seem to be shedding less. Cannot say how relieved I am to have finally found a food they like that is so good for them as well. It is pricey but if you want to save on costly vet bills down the road due to illness caused by poor nutrition, this is the way to go.",0.5938018327837162
B001EX9LGQ,Catfood,Very Healthy ingredients..compared to store brands..My<br />cats love this..they are Fussy Eaters..so I think many<br />cats would love this food..Will Buy it again.,0.5685298553173851
B003R0LKSG,Excellent Grain-Free Option,"We recently adopted a new kitten. We rotate Felidae grain-free canned formula with Wellness Turkey canned food. He loves both and eats well, he's never left a single whole pea behind. This is a very high-quality cat food, using excellent all-meat proteins. He is healthy, active,affectionate, and has a smooth glossy coat. Definitely worth the cost in the long run.<br /><br />I believe it is worthwhile to rotate different (high-quality) canned foods for variety and to not create a cat who will ONLY eat one brand. I think this is useful also in the event of a recall. I'm looking for a 3rd high-quality brand to add to the rotation.",0.567794291363927
B003SE19UK,Healthy & They LOVE It!,"My holistic vet recommended this, along with a few other brands. We tried them all, but my cats prefer this (especially the sardine version). The best part is their coats are so soft and clean and their eyes are so clear. AND (and I don't want to be rude, so I'll say this as delicately as I can) their waste is far less odorous than cats who eat the McDonalds junk found in most stores, which is a definite plus for me! The health benefits are so obvious - I highly recommend Holistic Select!",0.5660357548382248
B000VK6NDC,16 cats love it...,"I have 16 cats ranging from 1 to 11 yrs old and every single one of them likes this. I have tried Wellness, Holistic, Avoderm, Chicken Soup for the..., Merck Homestyle, BG (Before Grain), California Natural, and Evangers with mixed results. This is the first canned food that all of them will not only eat but eat all of it. I've fed them the Chicken and Liver flavor and the Seafood flavor so far and they seem to like both equally. I find it a bit more expensive than I had hoped it would be, but then again there is no waste and I consider that in itself a savings alone. I'm a firm believer in better ingredients and less fillers and allergens make for less poop, less litter use, fewer trips to the vet and an overall healthier cat.<br /><br />UPDATE: I have tried all the flavors on my cats and the chicken and liver seems to be the one they like the most. They have now officially rejected all the other flavors including the seafood fl

As you can see below we are able to search the data based on the semantics or meaning even if it doesnt contain the keywords

In [20]:
-- Assuming you have a stored procedure to get embeddings for a given text
declare @e vector(1536) 
exec dbo.get_embeddings @model = '<yourmodeldeploymentname>', @text = 'pet food', @embedding = @e output;

-- Hybrid query to find similar reviews based on vector similarity with simple filters
select top(10)
    f.Id,
    f.ProductId,
    f.UserId,
    f.Score,
    f.Summary,
    f.Text,
    f.combined,
    1-vector_distance('cosine', f.vectorbinary, @e) as similarity
from
    finefoodreviews f
where
    f.Score <= 2 -- Assuming you want to filter by low scores
order by
    vector_distance('cosine', f.vectorbinary, @e)

(10 rows affected)

Total execution time: 00:00:00.526

Id,ProductId,UserId,Score,Summary,Text,combined,similarity
4762,B00139TT72,A1CYGRMV5RB006,1,Doggy Prison Food,"I feed my dog Royal Canine brand which I typically have to mix with chicken stick or wet canned food for her to eat - I thought this stuff would be more flavorful and easy to serve. Upon opening I could tell I made a mistake - smells like cardboard - my dog won't eat it no matter how I dress it up, much less could I get her to eat it plain. Plus, it has none of the science behind it like higher end brands like Royal Canine - the Newman brand is spreading itself too thin here - they should stick with pasta sauce.","Title: Doggy Prison Food; Content: I feed my dog Royal Canine brand which I typically have to mix with chicken stick or wet canned food for her to eat - I thought this stuff would be more flavorful and easy to serve. Upon opening I could tell I made a mistake - smells like cardboard - my dog won't eat it no matter how I dress it up, much less could I get her to eat it plain. Plus, it has none of the science behind it like higher end brands like Royal Canine - the Newman brand is spreading itself too thin here - they should stick with pasta sauce.",0.5092674109459089
2949,B0002TJAZK,A22UFBP0KL3KQB,1,All By-Product & Corn -- Don't Waste Your Money!,"I made the mistake of believing that Science Diet was the best choice for my cat after the woman at the animal shelter mentioned it as the food they use. After doing some research I realized that Science Diet is no better than any grocery store generic brand or Iams (I would NOT recommend ANY of these options).<br /><br />The main ingredients are Chicken By-Product and Corn. Here are some interesting facts: Cat's can't digest corn...in fact, they are obligatory carnivores and only eat up until the point of reaching optimal nutrition. When I fed my cat the science diet he ate ALL THE TIME, constantly. After switching him over to Castor and Pollux Indoor Cat Formula he eats much less now.<br /><br />Also, By-Product means just that...beaks, feathers, toe-nail clippings, meat not suitable for human consumption, dead animals, etc. -- Totally not worth the $$$ ...it's cheaper just to dig in the trash and puree it yourself (doesn't sound so appealing does it??). Science Diet is notorious for implying superior quality products when they are in fact no better than all the other terrible pet food you find. Don't let your vet fool you either- vets usually get a 40% cut from Hill's to sell you their cat food.<br /><br />My recommendation is to use common sense and to do some research. Try brands like Newmans Own or Castor and Pollux as they have wholesome ingredients and a proven track record.<br /><br />I later called the shelter only to hear that they are required to feed the animals Science Diet as it is donated and they have a special arrangement with Petsmart to house their animals in their in store adoption centers. A requirement is to exclusively feed Science Diet to the animals.<br /><br />TRY THIS INSTEAD: <a href=""http://www.amazon.com/gp/product/B001BCUBZ0"">Castor & Pollux Ultramix Indoor Feline Fomula Dry Cat Food, 15-Pound Bag</a>","Title: All By-Product & Corn -- Don't Waste Your Money!; Content: I made the mistake of believing that Science Diet was the best choice for my cat after the woman at the animal shelter mentioned it as the food they use. After doing some research I realized that Science Diet is no better than any grocery store generic brand or Iams (I would NOT recommend ANY of these options).<br /><br />The main ingredients are Chicken By-Product and Corn. Here are some interesting facts: Cat's can't digest corn...in fact, they are obligatory carnivores and only eat up until the point of reaching optimal nutrition. When I fed my cat the science diet he ate ALL THE TIME, constantly. After switching him over to Castor and Pollux Indoor Cat Formula he eats much less now.<br /><br />Also, By-Product means just that...beaks, feathers, toe-nail clippings,

The below approach showcases the power of semantic search by finding reviews that are contextually related to the search query, even if they don’t contain exact match keywords. It also demonstrates how keyword search can be used in conjunction to ensure that certain words are present in the results

In [21]:
declare @e vector(1536) 
exec dbo.get_embeddings @model = '<yourmodeldeploymentname>', @text = 'help me find negative reviews for hot beverages', @embedding = @e output;

select top(10)
    f.Id,
    f.ProductId,
    f.UserId,
    f.Score,
    f.Summary,
    f.Text,
    f.combined,
    1-vector_distance('cosine', f.vectorbinary, @e) as similarity,
    case 
        when len(f.Text) > 100 then 'Detailed Review'
        else 'Short Review'
    end as ReviewLength,
    case 
        when f.Score >= 4 then 'High Score'
        when f.Score between 2 and 3 then 'Medium Score'
        else 'Low Score'
    end as ScoreCategory
from
    finefoodreviews f
where
     f.Score <= 2 -- Score threshold filter
    and f.Text like '%disappointing%' or f.Text like '%worst%' -- Inclusion of specific negative words
order by
    similarity desc, -- Order by similarity score
    f.Score desc, -- Secondary order by review score
    ReviewLength desc -- Tertiary order by review length


(10 rows affected)

Total execution time: 00:00:00.911

Id,ProductId,UserId,Score,Summary,Text,combined,similarity,ReviewLength,ScoreCategory
9239,B006N3IG4K,A7CTIXDLF8I5W,2,Either I got a bad batch or....,"...or the good reviewers' taste buds are weak. This is some disappointingly<br />weak coffee. I'm not some crazy ""gotta have my jet black sludge to get<br />through the day"" coffee drinker - I'd say my favorite (and my wife's) is<br />Caribou Blend. This blend is WEAK, it can't hold half-and-half or even<br />just whole milk without being wiped out. Now, I will say I brew it on<br />the ""large cup"" (9.25 oz, I believe) setting on our Keurig, so maybe it's<br />a bit better on the 8oz setting, or even the small setting. But Caribou (we<br />also like Donut Shop, Green Mountain Kona Blend 10%, etc.) can be easily<br />brewed on 9.25 and hold a good bold taste. This coffee doesn't<br />hold a CANDLE to any of those. Now I'm stuck with 40+ of these. Blech...maybe<br />the guests won't mind it! :)","Title: Either I got a bad batch or....; Content: ...or the good reviewers' taste buds are weak. This is some disappointingly<br />weak coffee. I'm not some crazy ""gotta have my jet black sludge to get<br />through the day"" coffee drinker - I'd say my favorite (and my wife's) is<br />Caribou Blend. This blend is WEAK, it can't hold half-and-half or even<br />just whole milk without being wiped out. Now, I will say I brew it on<br />the ""large cup"" (9.25 oz, I believe) setting on our Keurig, so maybe it's<br />a bit better on the 8oz setting, or even the small setting. But Caribou (we<br />also like Donut Shop, Green Mountain Kona Blend 10%, etc.) can be easily<br />brewed on 9.25 and hold a good bold taste. This coffee doesn't<br />hold a CANDLE to any of those. Now I'm stuck with 40+ of these. Blech...maybe<br />the guests won't mind it! :)",0.5143298503388504,Detailed Review,Medium Score
8530,B003VXFK44,A7CTIXDLF8I5W,2,Either I got a bad batch or....,"...or the good reviewers' taste buds are weak. This is some disappointingly<br />weak coffee. I'm not some crazy ""gotta have my jet black sludge to get<br />through the day"" coffee drinker - I'd say my favorite (and my wife's) is<br />Caribou Blend. This blend is WEAK, it can't hold half-and-half or even<br />just whole milk without being wiped out. Now, I will say I brew it on<br />the ""large cup"" (9.25 oz, I believe) setting on our Keurig, so maybe it's<br />a bit better on the 8oz setting, or even the small setting. But Caribou (we<br />also like Donut Shop, Green Mountain Kona Blend 10%, etc.) can be easily<br />brewed on 9.25 and hold a good bold taste. This coffee doesn't<br />hold a CANDLE to any of those. Now I'm stuck with 40+ of these. Blech...maybe<br />the guests won't mind it! :)","Title: Either I got a bad batch or....; Content: ...or the good reviewers' taste buds are weak. This is some disappointingly<br />weak coffee. I'm not some crazy ""gotta have my jet black sludge to get<br />through the day"" coffee drinker - I'd say my favorite (and my wife's) is<br />Caribou Blend. This blend is WEAK, it can't hold half-and-half or even<br />just whole milk without being wiped out. Now, I will say I brew it on<br />the ""large cup"" (9.25 oz, I believe) setting on our Keurig, so maybe it's<br />a bit better on the 8oz setting, or even the small setting. But Caribou (we<br />also like Donut Shop, Green Mountain Kona Blend 10%, etc.) can be easily<br />brewed on 9.25 and hold a good bold taste. This coffee doesn't<br />hold a CANDLE to any of those. Now I'm stuck with 40+ of these. Blech...maybe<br />the guests won't mind it! :)",0.5143298503388504,Detailed Review,Medium Score
3327,B005K4Q1VI,ANTA6BNDLCQP5,1,NASTY,"Since I am willing to try alot of new things. I thought that I would buy this product.. And since I do like hot cocoa, I bought 2 boxes.. WORST thing I could have done.. This is bland, with very little flavour. It also has a grainy texture near the bottom of the mug.<br />Personally I will not buy this product a